贝叶斯分类：对于给定数据，判断其属于哪个类别。通过计算所有p（Ci|x,y)，概率最大对应类别就是分类结果。计算过程中用到贝叶斯规则，将计算转化为P（x,y|Ci),p(Ci)计算。
之所以称为朴素：即假设所有属性概率上条件独立

In [17]:
import numpy as np

def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]    #1 is abusive, 0 not
    return postingList, classVec

def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

def trainNB0(trainMatrix, trainCategory):
    trainData = np.array(trainMatrix)
    trainLabels = np.array(trainCategory)
    #get number of documents
    numTrainDocs = trainData.shape[0]
    numWords = trainData.shape[1]
    #probability of abusive
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords) 
    
    #to avoid divided by zero
    p0Denom = 2.0
    p1Denom = 2.0  
    
    #count abusive data, sum all data which label is 1(abusive) along axis = 0
    p1Num = np.sum(trainData[trainLabels == 1,:], axis=0, keepdims=True)
    p1Denom = np.sum(trainData[trainLabels == 1,:])
    #count non-abusive data, sum all data which label is 0(abusive) along axis = 0
    p0Num = np.sum(trainData[trainLabels == 0,:], axis=0, keepdims=True)
    p0Denom = np.sum(trainData[trainLabels == 0,:])

    #calculate p(wi|1) and p(wi|0)
    p1Vect = np.log(p1Num/p1Denom)          
    p0Vect = np.log(p0Num/p0Denom)          
    return p0Vect, p1Vect, pAbusive

def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0
def testingNB():
    #load data
    listOposts, listClasses = loadDataSet()
    #create vocab list
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    #create training matrix
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    #calculate condition probabilities and prior probiblity
    p0V, p1V, pAb = trainNB0(trainMat, listClasses)
    
    #create test data array
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as : %d' % classifyNB(thisDoc, p0V, p1V, pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as : %d' % classifyNB(thisDoc, p0V, p1V, pAb))
    
def bagOfWords2VecMN(vocabList, inputSet):
    """
    Arguments:
        vocabList -- list of vocabulary
        inputSet  -- input words set
    Return:
        returnVec -- a vector of all words count
    """
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            #if word of inputSet exists in vocabList, count++ this word
            returnVec[vocabList.index[word]] += 1
    return returnVec

def textParse(bigString):
    """
    Arguments:
        bigString -- long string
    Return:
        list
        delete len < 2 string and change all string to lowercase
    """
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

def spamTest():
    #new three empty list
    docList = []
    classList = []
    fullText = []
    #load spam and non spam emails , 25 each
    for i in range(1, 26):
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    trainingSet = range(50)
    testSEt = []
    #randomly choose 10 samples from 50 train samples as testset
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    
    # the left samples as training set
    trainMat = []
    trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    
    p0V, p1V, pAb = trainNB0(np.array(trainMat), np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V, pAb) != classlist[docIndex]:
            errorCount += 1
    print('the error rate is %f '% float(errorCount)/len(testSet))

def calcMostFreq(vocabList, fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.count(token)
    sortedFreq = sorted(freqDict.items(), key=operator.itemgetter(1), reverse=True)
    return sortedFreq[:30]

def localWords(feed1, feed0):
    import feedparser
    docList = []; classList = []; fullText = []
    minLen = min(len(feed1['entries']), len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1) #NY is class 1
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    top30Words = calcMostFreq(vocabList, fullText)   #remove top 30 words
    for pairW in top30Words:
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
    trainingSet = range(2*minLen); testSet = []           #create test set
    for i in range(20):
        randIndex = int(np.random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(list(trainingSet)[randIndex])
    trainMat = []; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is: ', float(errorCount)/len(testSet))
    return vocabList, p0V, p1V

def getTopWords(ny, sf):
    import operator
    vocabList, p0V, p1V = localWords(ny, sf)
    topNY = []; topSF = []
    for i in range(len(p0V)):
        if p0V[i] > -6.0: topSF.append((vocabList[i], p0V[i]))
        if p1V[i] > -6.0: topNY.append((vocabList[i], p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    for item in sortedSF:
        print(item[0])
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sortedNY:
        print(item[0])

testingNB()

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
2.0
[1. 1. 2. 1. 1. 2. 1. 1. 1. 1. 2. 1. 2. 1. 1. 1. 2. 1. 1. 1. 1. 2. 1. 1.
 1. 1. 1. 2. 1. 1. 2. 1.]
10.0
[1. 1. 2. 1. 1. 2. 1. 1. 1. 1. 2. 1. 2. 1. 1. 1. 2. 1. 1. 1. 1. 2. 1. 1.
 1. 1. 1. 2. 1. 1. 2. 1.]
10.0
[1. 1. 2. 1. 1. 2. 1. 1. 1. 1. 2. 1. 2. 1. 1. 1. 2. 2. 1. 2. 1. 3. 1. 1.
 1. 2. 2. 2. 1. 1. 2. 1.]
15.0
[1. 1. 2. 1. 1. 2. 1. 1. 1. 1. 2. 1. 2. 1. 1. 1. 2. 2. 1. 2. 1. 3. 1. 1.
 1. 2. 2. 2. 1. 1. 2. 1.]
15.0
[1. 1. 3. 1. 2. 2. 1. 1. 1. 2. 2. 1. 2. 1. 2. 1. 2. 2. 1. 2. 1. 4. 1. 1.
 1. 3. 2. 2. 1. 1. 2. 1.]
21.0
['love', 'my', 'dalmation'] classified as : 0
['stupid', 'garbage'] classified as : 1
